In [14]:
import numpy as np
from astropy.io import fits
from scipy.signal import fftconvolve
from scipy.fftpack import fft2, ifft2, fftshift
import stpsf
import astropy.units as u
import os
## Trying to correctly pass the header:
import glob
from astropy.wcs import WCS
from reproject import reproject_interp

In [15]:

# --- paths ---
#The dir names might confuse but i am first regridding and then will smooth
#I dont have time to change all dirs so i am just using old

smooth_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic"
out_dir = "../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid"
os.makedirs(out_dir, exist_ok=True)

# Reference cube (Fe II or ch4)
ref_path = os.path.join(smooth_dir, "NGC253_PIII_17.9.line.fits")
with fits.open(ref_path) as ref_hdul:
    ref_hdr = ref_hdul[0].header.copy()
    ref_data = ref_hdul[0].data.copy()
ref_wcs_3d = WCS(ref_hdr)
ref_wcs_2d = ref_wcs_3d.dropaxis(2)
ref_hdr_2d = ref_wcs_2d.to_header()
ny_ref, nx_ref = ref_data.shape[1], ref_data.shape[2]
shape_out_ref = (ny_ref, nx_ref)

# --- Input cubes ---
cube_files = sorted(glob.glob(os.path.join(smooth_dir, "*.fits")))
print(f"Found {len(cube_files)} smoothed cubes.")

for infile in cube_files:
    name = os.path.basename(infile)
    outfile = os.path.join(out_dir, name.replace(".fits", "_regrid.fits"))

    # --- Read input cube ---
    data_in = fits.getdata(infile)
    hdr_in = fits.getheader(infile)
    wcs_in_3d = WCS(hdr_in)
    wcs_in_2d = wcs_in_3d.dropaxis(2)
    hdr_in_2d = wcs_in_2d.to_header()

    nlam = data_in.shape[0]
    print(f"Regridding {name}: {nlam} slices -> {shape_out_ref}")

    # --- Reproject slice by slice ---
    regridded = np.empty((nlam, ny_ref, nx_ref), dtype=np.float32)
    for i in range(nlam):
        plane = data_in[i].astype(float)
        plane_masked = np.where(np.isfinite(plane), plane, np.nan)
        out_plane, _ = reproject_interp((plane_masked, hdr_in_2d),
                                        ref_hdr_2d, shape_out=shape_out_ref)
        regridded[i] = out_plane

    # --- Build output header ---
    out_hdr = ref_hdr.copy()
    out_hdr["NAXIS3"] = nlam
    for key in hdr_in:
        if "3" in key or key.startswith("REST") or key.startswith("SPECSYS"):
            out_hdr[key] = hdr_in[key]



    fits.writeto(outfile, regridded, out_hdr, overwrite=True)
    print(f"  → wrote {outfile} shape={regridded.shape}")

print("All smoothed cubes regridded successfully.")



Found 100 smoothed cubes.
Regridding NGC253_ArIII_9.0.cont.fits: 53 slices -> (93, 75)


Set OBSGEO-B to    21.635079 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821546.245 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    21.635079 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821546.245 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    21.635080 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821525.558 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid/NGC253_ArIII_9.0.cont_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArIII_9.0.line.fits: 53 slices -> (93, 75)


Set OBSGEO-B to    21.635079 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821546.245 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    21.635080 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821525.558 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid/NGC253_ArIII_9.0.line_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArII_7.0.cont.fits: 67 slices -> (93, 75)


...

  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid/NGC253_U_5.65_5.7.line_regrid.fits shape=(54, 93, 75)
Regridding NGC253_U_5.74_5.7.cont.fits: 54 slices -> (93, 75)


Set OBSGEO-B to    21.635079 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821546.245 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    21.635080 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821525.558 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid/NGC253_U_5.74_5.7.cont_regrid.fits shape=(54, 93, 75)
Regridding NGC253_U_5.74_5.7.line.fits: 54 slices -> (93, 75)


Set OBSGEO-B to    21.635079 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821546.245 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    21.635080 from OBSGEO-[XYZ].
Set OBSGEO-H to 1548821525.558 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid/NGC253_U_5.74_5.7.line_regrid.fits shape=(54, 93, 75)
All smoothed cubes regridded successfully.


In [16]:


# --------------------------------------------------
# Band selection
# --------------------------------------------------
def band_from_wavelength(lam):
    lam = float(lam)
    if 4.90 <= lam <= 5.74:   return "1A"
    if 5.66 <= lam <= 6.63:   return "1B"
    if 6.53 <= lam <= 7.65:   return "1C"
    if 7.51 <= lam <= 8.77:   return "2A"
    if 8.67 <= lam <= 10.13:  return "2B"
    if 10.01 <= lam <= 11.70: return "2C"
    if 11.55 <= lam <= 13.47: return "3A"
    if 13.34 <= lam <= 15.57: return "3B"
    if 15.41 <= lam <= 17.98: return "3C"
    raise ValueError(f"Wavelength {lam} µm outside MIRI MRS range")



# --------------------------------------------------
# Kernel construction (Wiener deconvolution)
# --------------------------------------------------
def compute_matching_kernel(psf_native, psf_target, reg=1e-6):
    """
    Compute kernel K such that psf_native * K ≈ psf_target.
    Both input PSFs must have the same shape.
    """
    Fn = fft2(psf_native)
    Ft = fft2(psf_target)
    denom = np.abs(Fn)**2 + reg
    K = np.real(ifft2(np.conj(Fn) * Ft / denom))
    K = fftshift(K)
    K /= K.sum()          # flux conservation
    return K

# --------------------------------------------------
# Rest‑frame wavelengths for each line (from file names)
# --------------------------------------------------

rest_um = {
    "S9_5_0":    4.95,
    "S8_5_1":    5.05,
    "H6d_5_1":   5.13,
    "FeII_5_3":  5.34,
    "H7k_5_4":   5.38,
    "S7_5_5":    5.51,
    "H7i_5_5":   5.53,
    "MgV_5_6":   5.60,
    "H7t_5_7":   5.71,
    "S7_1-1_5_8":5.81,
    "H6g_5_9":   5.91,
    "H7e_6_0":   5.96,
    "S6_6_1":    6.11,
    "H7z_6_3":   6.29,
    "NiII_6_6":  6.64,
    "He_6_7":    6.72,
    "H7e_6_8":   6.77,
    "S5_6_9":    6.91,
    "ArII_7_0":  6.99,
    "NiIII_7_3": 7.35,
    "H5a_7_5":   7.46,
    "H5b_7_5":   7.50,
    "H7d_7_5":   7.51,
    "NeVI_7_7":  7.65,
    "H8t_7_8":   7.78,
    "S4_8_0":    8.03,
    "H7g_8_8":   8.76,
    "ArIII_9_0": 8.99,
    "S3_9_7":    9.66,
    "SIV_10_5":  10.51,
    "NiII_10_7": 10.68,
    "H7b_11_3":  11.31,
    "S2_12_3":   12.28,
    "H6a_12_4":  12.37,
    "H8g_12_4":  12.39,
    "NeII_12_8": 12.81,
    "MgV_13_5": 13.54,
    "C2H2_13_7": 13.70,
    "HCN_14_0": 14.0,
    "NeV_14_3":  14.32,
    "ClII_14_4": 14.37,
    "CO2_15_0": 15.0,
    "NeIII_15_6":15.56,
    "H8b_16_2":  16.21,
    "S1_17_0":   17.03,
    "PIII_17_9": 17.88,
}



def get_line_wavelength_from_dict(filename, rest_um):
    base = os.path.basename(filename)
    if base.startswith("NGC253_"):
        base = base.replace("NGC253_", "")
    base = base.replace(".line_regrid.fits", "")
    name_key = base.replace(".", "_")
    if name_key in rest_um:
        return rest_um[name_key]
    raise RuntimeError(f"Line '{name_key}' not found in rest_um dictionary (file: {filename})")

# --------------------------------------------------
# User parameters
# --------------------------------------------------
IN_DIR   ="../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_regrid"
OUT_DIR  = "../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth"
TARGET_LAMBDA = 17.8846      # µm
TARGET_BAND   = "3C"
FILE_GLOB = os.path.join(IN_DIR, "*.line_regrid.fits")

os.makedirs(OUT_DIR, exist_ok=True)
files = sorted(glob.glob(FILE_GLOB))

# --------------------------------------------------
# Main processing loop
# --------------------------------------------------
skipped_lines = []

for infile in files:
    name = os.path.basename(infile)
    print(f"\nProcessing {name}")

    with fits.open(infile) as hdul:
        flux = hdul[0].data
        hdr  = hdul[0].header
        # --- Pixel scale for THIS file (arcsec) ---
        pixscale = abs(hdr["CDELT2"]) * 3600.0

    # --- Determine line wavelength and band ---
    try:
        lam_line = get_line_wavelength_from_dict(infile, rest_um)
        band_line = band_from_wavelength(lam_line)
    except RuntimeError as e:
        print(f"  ⚠ Skipping {name}: {e}")
        skipped_lines.append(name)
        continue

    print(f"  Line λ = {lam_line:.4f} µm  |  Band = {band_line}")

    # --- Target PSF (computed for this file's pixel scale) ---
    miri_target = stpsf.MIRI()
    miri_target.mode = "IFU"
    miri_target.band = TARGET_BAND
    miri_target.pixelscale = pixscale #0.20
    psf_target = miri_target.calc_psf(
        monochromatic=TARGET_LAMBDA * u.micron
    )[3].data
    psf_target /= psf_target.sum()

    # --- Native PSF (at line wavelength, same pixel scale) ---
    miri_native = stpsf.MIRI()
    miri_native.mode = "IFU"
    miri_native.band = band_line
    miri_native.pixelscale = pixscale
    psf_native = miri_native.calc_psf(
        monochromatic=lam_line * u.micron
    )[3].data
    psf_native /= psf_native.sum()

    # --- Pad both PSFs to same size (for FFT) ---
    h = max(psf_native.shape[0], psf_target.shape[0])
    w = max(psf_native.shape[1], psf_target.shape[1])

    def pad(psf):
        out = np.zeros((h, w))
        y = (h - psf.shape[0]) // 2
        x = (w - psf.shape[1]) // 2
        out[y:y+psf.shape[0], x:x+psf.shape[1]] = psf
        return out

    psf_native_p = pad(psf_native)
    psf_target_p = pad(psf_target)

    # --- Compute matching kernel ---
    kernel = compute_matching_kernel(psf_native_p, psf_target_p, reg=1e-6)

    # --- Convolve each slice (2D image) ---
    sm_flux = np.zeros_like(flux)
    for i in range(flux.shape[0]):
        img = np.nan_to_num(flux[i], nan=0.0)
        mask = ~np.isnan(flux[i])
        sm = fftconvolve(img, kernel, mode="same")
        sm[~mask] = np.nan
        sm_flux[i] = sm

    # --- Write output (flux only, no error extension) ---
    outname = os.path.join(OUT_DIR, name.replace(".fits", "_smooth.fits"))
    fits.writeto(outname, sm_flux, hdr, overwrite=True)
    print(f"  → Saved {outname}")

# --------------------------------------------------
# Summary of skipped files
# --------------------------------------------------
if skipped_lines:
    print("\n⚠ The following line files were skipped (not in rest_um dictionary):")
    for f in skipped_lines:
        print("   ", f)

print("\nAll available line maps PSF-homogenized correctly.")


Processing NGC253_ArIII_9.0.line_regrid.fits
  Line λ = 8.9900 µm  |  Band = 2B


/Users/u826s018/miniconda3/envs/nearbygalaxies/lib/python3.11/site-packages/stpsf/opds.py:1759: UserWarning: For (V2,V3) = [-8.399315 -5.319645] arcmin, Field point -8.254199999999999 arcmin, -2.480355000000001 arcmin not within valid region for field dependence model of OTE WFE for MIRI: -8.254199999999999 arcmin--6.21738 arcmin, -2.557224 arcmin--0.5632056 arcmin.  Clipping to closest available valid location, 0.14511500000000055 arcmin away from the requested coordinates.
  warnings.warn(warning_message)
/Users/u826s018/miniconda3/envs/nearbygalaxies/lib/python3.11/site-packages/stpsf/opds.py:1759: UserWarning: For (V2,V3) = [-8.39228333 -5.32616667] arcmin, Field point -8.254199999999999 arcmin, -2.473833333333334 arcmin not within valid region for field dependence model of OTE WFE for MIRI: -8.254199999999999 arcmin--6.21738 arcmin, -2.557224 arcmin--0.5632056 arcmin.  Clipping to closest available valid location, 0.13808333333333422 arcmin away from the requested coordinates.
  w

  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_ArIII_9.0.line_regrid_smooth.fits

Processing NGC253_ArII_7.0.line_regrid.fits
  Line λ = 6.9900 µm  |  Band = 1C


/Users/u826s018/miniconda3/envs/nearbygalaxies/lib/python3.11/site-packages/stpsf/opds.py:1759: UserWarning: For (V2,V3) = [-8.38815333 -5.31410167] arcmin, Field point -8.254199999999999 arcmin, -2.485898333333334 arcmin not within valid region for field dependence model of OTE WFE for MIRI: -8.254199999999999 arcmin--6.21738 arcmin, -2.557224 arcmin--0.5632056 arcmin.  Clipping to closest available valid location, 0.13395333333333426 arcmin away from the requested coordinates.
  warnings.warn(warning_message)


  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_ArII_7.0.line_regrid_smooth.fits

Processing NGC253_C2H2_13.7.line_regrid.fits
  Line λ = 13.7000 µm  |  Band = 3B


...

  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H6g_5.9.line_regrid_smooth.fits

Processing NGC253_H7b_11.3.line_regrid.fits
  Line λ = 11.3100 µm  |  Band = 2C


/Users/u826s018/miniconda3/envs/nearbygalaxies/lib/python3.11/site-packages/stpsf/opds.py:1759: UserWarning: For (V2,V3) = [-8.38974833 -5.32495833] arcmin, Field point -8.254199999999999 arcmin, -2.4750416666666677 arcmin not within valid region for field dependence model of OTE WFE for MIRI: -8.254199999999999 arcmin--6.21738 arcmin, -2.557224 arcmin--0.5632056 arcmin.  Clipping to closest available valid location, 0.13554833333333427 arcmin away from the requested coordinates.
  warnings.warn(warning_message)


  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7b_11.3.line_regrid_smooth.fits

Processing NGC253_H7d_7.5.line_regrid.fits
  Line λ = 7.5100 µm  |  Band = 1C
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7d_7.5.line_regrid_smooth.fits

Processing NGC253_H7e_6.0.line_regrid.fits
  Line λ = 5.9600 µm  |  Band = 1B
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7e_6.0.line_regrid_smooth.fits

Processing NGC253_H7e_6.8.line_regrid.fits
  Line λ = 6.7700 µm  |  Band = 1C
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7e_6.8.line_regrid_smooth.fits

Processing NGC253_H7g_8.8.line_regrid.fits
  Line λ = 8.7600 µm  |  Band = 2A


/Users/u826s018/miniconda3/envs/nearbygalaxies/lib/python3.11/site-packages/stpsf/opds.py:1759: UserWarning: For (V2,V3) = [-8.39108833 -5.32144667] arcmin, Field point -8.254199999999999 arcmin, -2.478553333333333 arcmin not within valid region for field dependence model of OTE WFE for MIRI: -8.254199999999999 arcmin--6.21738 arcmin, -2.557224 arcmin--0.5632056 arcmin.  Clipping to closest available valid location, 0.13688833333333505 arcmin away from the requested coordinates.
  warnings.warn(warning_message)


  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7g_8.8.line_regrid_smooth.fits

Processing NGC253_H7i_5.5.line_regrid.fits
  Line λ = 5.5300 µm  |  Band = 1A
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7i_5.5.line_regrid_smooth.fits

Processing NGC253_H7k_5.4.line_regrid.fits
  Line λ = 5.3800 µm  |  Band = 1A
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7k_5.4.line_regrid_smooth.fits

Processing NGC253_H7t_5.7.line_regrid.fits
  Line λ = 5.7100 µm  |  Band = 1A
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7t_5.7.line_regrid_smooth.fits

Processing NGC253_H7z_6.3.line_regrid.fits
  Line λ = 6.2900 µm  |  Band = 1B
  → Saved ../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth/NGC253_H7z_6.3.line_regrid_smooth.fits

Processing NGC253_H8b_16.2.line_r

In [17]:

# ---- Paths ----
input_dir = "../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_smooth"         #use this for frequency based x axis
output_dir = "../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/regrid_V2/line_masked"

os.makedirs(output_dir, exist_ok=True)

# ---- Reference Ch I cube for mask ----
ch1_ref = os.path.join(input_dir, "NGC253_S9_5.0.line_regrid_smooth.fits")         #for frequency based maps
if not os.path.exists(ch1_ref):
    raise FileNotFoundError(f"Ch 1 reference cube not found: {ch1_ref}")

fe_data = fits.getdata(ch1_ref)
if fe_data.ndim != 3:
    raise RuntimeError(f"Ch I cube not 3D: shape={fe_data.shape}")

mask = np.ones_like(fe_data[0], dtype=float)
mask[~np.isfinite(fe_data[0])] = np.nan
print(f"✅ Mask built from Ch I first plane: shape={mask.shape}")

# ---- Apply mask to each cube ----
for fname in sorted(os.listdir(input_dir)):
    if not fname.endswith(".fits"):
        continue
    infile = os.path.join(input_dir, fname)
    data = fits.getdata(infile)
    hdr = fits.getheader(infile)

    if data.ndim != 3:
        print(f"Skipping {fname} (not 3D)")
        continue

    # Apply mask: NaN where Ch I is invalid
    data_masked = data * mask[np.newaxis, :, :]

    # Write masked cube
    outfile = os.path.join(output_dir, fname.replace(".fits", "_maskch1.fits"))
    fits.writeto(outfile, data_masked, hdr, overwrite=True)
    print(f"→ Masked {fname}  →  {os.path.basename(outfile)}")

print("🎉 All cubes masked using ch1 S9 reference.")


✅ Mask built from Ch I first plane: shape=(93, 75)
→ Masked NGC253_ArIII_9.0.line_regrid_smooth.fits  →  NGC253_ArIII_9.0.line_regrid_smooth_maskch1.fits
→ Masked NGC253_ArII_7.0.line_regrid_smooth.fits  →  NGC253_ArII_7.0.line_regrid_smooth_maskch1.fits
→ Masked NGC253_C2H2_13.7.line_regrid_smooth.fits  →  NGC253_C2H2_13.7.line_regrid_smooth_maskch1.fits
→ Masked NGC253_CO2_15.0.line_regrid_smooth.fits  →  NGC253_CO2_15.0.line_regrid_smooth_maskch1.fits
→ Masked NGC253_ClII_14.4.line_regrid_smooth.fits  →  NGC253_ClII_14.4.line_regrid_smooth_maskch1.fits
→ Masked NGC253_FeII_5.3.line_regrid_smooth.fits  →  NGC253_FeII_5.3.line_regrid_smooth_maskch1.fits
→ Masked NGC253_H5a_7.5.line_regrid_smooth.fits  →  NGC253_H5a_7.5.line_regrid_smooth_maskch1.fits
→ Masked NGC253_H5b_7.5.line_regrid_smooth.fits  →  NGC253_H5b_7.5.line_regrid_smooth_maskch1.fits
→ Masked NGC253_H6a_12.4.line_regrid_smooth.fits  →  NGC253_H6a_12.4.line_regrid_smooth_maskch1.fits
→ Masked NGC253_H6d_5.1.line_regrid_sm

In [18]:
raise Exception('Stop Here')

Exception: Stop Here

In [ ]:
### One calculation
'''
import numpy as np
from astropy.io import fits
from scipy.signal import fftconvolve
from scipy.fftpack import fft2, ifft2, fftshift
import stpsf
import astropy.units as u
import os
from glob import glob
import re


# --------------------------------------------------
# Band selection
# --------------------------------------------------
def band_from_wavelength(lam):
    lam = float(lam)
    if 4.90 <= lam <= 5.74:   return "1A"
    if 5.66 <= lam <= 6.63:   return "1B"
    if 6.53 <= lam <= 7.65:   return "1C"
    if 7.51 <= lam <= 8.77:   return "2A"
    if 8.67 <= lam <= 10.13:  return "2B"
    if 10.01 <= lam <= 11.70: return "2C"
    if 11.55 <= lam <= 13.47: return "3A"
    if 13.34 <= lam <= 15.57: return "3B"
    if 15.41 <= lam <= 17.98: return "3C"
    raise ValueError(f"Wavelength {lam} µm outside MIRI MRS range")


# --------------------------------------------------
# Kernel construction
# --------------------------------------------------
def compute_matching_kernel(psf_native, psf_target, reg=1e-6):
    Fn = fft2(psf_native)
    Ft = fft2(psf_target)
    denom = np.abs(Fn)**2 + reg
    K = np.real(ifft2(np.conj(Fn) * Ft / denom))
    K = fftshift(K)
    K /= K.sum()
    return K



rest_um = {
    "S9_5_0":    4.95,
    "S8_5_1":    5.05,
    "H6d_5_1":   5.13,
    "FeII_5_3":  5.34,
    "H7k_5_4":   5.38,
    "S7_5_5":    5.51,
    "H7i_5_5":   5.53,
    "MgV_5_6":   5.60,
    "H7t_5_7":   5.71,
    "S7_1-1_5_8":5.81,
    "H6g_5_9":   5.91,
    "H7e_6_0":   5.96,
    "S6_6_1":    6.11,
    "H7z_6_3":   6.29,
    "NiII_6_6":  6.64,
    "He_6_7":    6.72,
    "H7e_6_8":   6.77,
    "S5_6_9":    6.91,
    "ArII_7_0":  6.99,
    "NiIII_7_3": 7.35,
    "H5a_7_5":   7.46,
    "H5b_7_5":   7.50,
    "H7d_7_5":   7.51,
    "NeVI_7_7":  7.65,
    "H8t_7_8":   7.78,
    "S4_8_0":    8.03,
    "H7g_8_8":   8.76,
    "ArIII_9_0": 8.99,
    "S3_9_7":    9.66,
    "SIV_10_5":  10.51,
    "NiII_10_7": 10.68,
    "H7b_11_3":  11.31,
    "S2_12_3":   12.28,
    "H6a_12_4":  12.37,
    "H8g_12_4":  12.39,
    "NeII_12_8": 12.81,
    "MgV_13_5": 13.54,
    "C2H2_13_7": 13.70,
    "HCN_14_0": 14.0,
    "NeV_14_3":  14.32,
    "ClII_14_4": 14.37,
    "CO2_15_0": 15.0,
    "NeIII_15_6":15.56,
    "H8b_16_2":  16.21,
    "S1_17_0":   17.03,
    "PIII_17_9": 17.88,
}


# --------------------------------------------------
# Extract wavelength from line list
# --------------------------------------------------
def get_line_wavelength_from_dict(filename, rest_um):
    base = os.path.basename(filename)

    # Remove galaxy prefix
    if base.startswith("NGC253_"):
        base = base.replace("NGC253_", "")

    # Remove only .line.fits
    base = base.replace(".line.fits", "")

    # Convert decimal → underscore for dict key
    name_key = base.replace(".", "_")

    if name_key in rest_um:
        return rest_um[name_key]

    raise RuntimeError(
        f"Line '{name_key}' not found in rest_um dictionary (file: {filename})"
    )


# --------------------------------------------------
# USER PARAMETERS
# --------------------------------------------------
IN_DIR  = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic"
OUT_DIR = "../../github/astr796_25_V2/NGC253_output/3channel/jwst_psf/line_smooth"

TARGET_LAMBDA = 17.8846  # µm
TARGET_BAND   = "3C"

FILE_GLOB = os.path.join(IN_DIR, "*.line.fits")
# --------------------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)
files = sorted(glob(FILE_GLOB))

# --- Pixel scale from first cube ---
with fits.open(files[0]) as hdul:
    pixscale = abs(hdul[0].header["CDELT2"]) * 3600.0

# --- Target PSF (once) ---
miri_target = stpsf.MIRI()
miri_target.mode = "IFU"
miri_target.band = TARGET_BAND
miri_target.pixelscale = pixscale
psf_target = miri_target.calc_psf(
    monochromatic=TARGET_LAMBDA * u.micron
)[1].data
psf_target /= psf_target.sum()

# --------------------------------------------------
# Main loop
# --------------------------------------------------
# List to store skipped lines
skipped_lines = []

# --------------------------------------------------
# Main loop
# --------------------------------------------------
for infile in files:
    name = os.path.basename(infile)
    print(f"\nProcessing {name}")

    with fits.open(infile) as hdul:
        flux = hdul[0].data
        hdr  = hdul[0].header

    # --- Determine line wavelength and band ---
    try:
        lam_line = get_line_wavelength_from_dict(infile, rest_um)
        band_line = band_from_wavelength(lam_line)
    except RuntimeError as e:
        print(f"  ⚠ Skipping {name}: {e}")
        skipped_lines.append(name)
        continue  # skip to next file

    print(f"  Line λ = {lam_line:.4f} µm  |  Band = {band_line}")

    # --- Native PSF ---
    miri_native = stpsf.MIRI()
    miri_native.mode = "IFU"
    miri_native.band = band_line
    miri_native.pixelscale = pixscale
    psf_native = miri_native.calc_psf(
        monochromatic=lam_line * u.micron
    )[1].data
    psf_native /= psf_native.sum()

    # --- Pad PSFs ---
    h = max(psf_native.shape[0], psf_target.shape[0])
    w = max(psf_native.shape[1], psf_target.shape[1])

    def pad(psf):
        out = np.zeros((h, w))
        y = (h - psf.shape[0]) // 2
        x = (w - psf.shape[1]) // 2
        out[y:y+psf.shape[0], x:x+psf.shape[1]] = psf
        return out

    psf_native_p = pad(psf_native)
    psf_target_p = pad(psf_target)

    # --- Kernel ---
    kernel = compute_matching_kernel(psf_native_p, psf_target_p)

    # --- Convolve ---
    sm_flux = np.zeros_like(flux)

    for i in range(flux.shape[0]):
        img = np.nan_to_num(flux[i], nan=0.0)
        mask = ~np.isnan(flux[i])
        sm = fftconvolve(img, kernel, mode="same")
        sm[~mask] = np.nan
        sm_flux[i] = sm
    
    # --- Write output ---
    outname = os.path.join(
        OUT_DIR,
        name.replace(".fits", "_smooth.fits")
    )
    fits.writeto(outname, sm_flux, hdr, overwrite=True)
    print(f"  → Saved {outname}")

# --------------------------------------------------
# Summary of skipped lines
# --------------------------------------------------
if skipped_lines:
    print("\n⚠ The following line files were skipped (not in rest_um dictionary):")
    for f in skipped_lines:
        print("   ", f)

print("\nAll available line cubes PSF-homogenized correctly.")
'''

In [ ]:
import numpy as np
from astropy.io import fits
from spectral_cube import SpectralCube
from scipy.signal import fftconvolve
from scipy.fftpack import fft2, ifft2, fftshift
import stpsf
import astropy.units as u
import os


def band_from_wavelength(lam):
    """Return MIRI MRS band for a wavelength in µm."""
    lam = float(lam)
    # Channel 1
    if 4.90 <= lam <= 5.74:  return "1A"   # SHORT
    if 5.66 <= lam <= 6.63:  return "1B"   # MEDIUM
    if 6.53 <= lam <= 7.65:  return "1C"   # LONG

    # Channel 2
    if 7.51 <= lam <= 8.77:  return "2A"   # SHORT
    if 8.67 <= lam <= 10.13: return "2B"   # MEDIUM
    if 10.01 <= lam <= 11.70: return "2C"  # LONG

    # Channel 3
    if 11.55 <= lam <= 13.47: return "3A"  # SHORT
    if 13.34 <= lam <= 15.57: return "3B"  # MEDIUM
    if 15.41 <= lam <= 17.98: return "3C"  # LONG

    raise ValueError(f"Wavelength {lam} µm is outside MIRI MRS range")


def compute_matching_kernel(psf_native, psf_target, reg=1e-6):
    """
    Compute kernel K such that psf_native * K ≈ psf_target.
    Uses FFT deconvolution with a small regularisation.
    Both psf_native and psf_target must be 2D arrays of the same shape.
    Returns K (same shape) normalised to sum = 1.
    """
    # Ensure same shape; pad if necessary (here we assume they are same)
    if psf_native.shape != psf_target.shape:
        raise ValueError("PSFs must have the same shape")
    
    # FFT
    Fn = fft2(psf_native)
    Ft = fft2(psf_target)
    
    # Wiener deconvolution: K_hat = conj(Fn) * Ft / (|Fn|^2 + reg)
    # To avoid complex division, we compute the complex ratio directly with regularisation.
    denom = np.abs(Fn)**2 + reg
    K_hat = np.conj(Fn) * Ft / denom
    
    # Inverse FFT
    K = np.real(ifft2(K_hat))
    
    # Shift to centre (optional, but good for visualisation)
    K = fftshift(K)
    
    # Normalise to sum=1 (flux conservation)
    K /= K.sum()
    
    return K

def smooth_cube_to_target_psf(infile, outfile, target_wavelength=17.98):
    """
    Homogenise each slice to the PSF at target_wavelength.
    For each slice, compute the matching kernel via FFT deconvolution
    of the native PSF (at slice wavelength) and the target PSF.
    """
    # Read cube
    sc = SpectralCube.read(infile, hdu=1)
    flux = fits.getdata(infile, hdu=1)
    err  = fits.getdata(infile, hdu=2)
    sci_hdr = sc.header.copy()

    # Build wavelength array
    if all(k in sci_hdr for k in ["CRVAL3", "CDELT3", "CRPIX3"]):
        nlam = sci_hdr["NAXIS3"]
        lambdas = ((np.arange(nlam) - (sci_hdr["CRPIX3"] - 1)) *
                   sci_hdr["CDELT3"] + sci_hdr["CRVAL3"])
    else:
        raise ValueError("Spectral WCS keywords missing from header!")

    # Pixel scale in arcsec
    pixscale = sci_hdr["CDELT2"] * 3600.0

    # Allocate output arrays
    sm_flux = np.empty_like(flux)
    sm_err  = np.empty_like(err)

    # --- Pre‑compute target PSF (once) ---
    target_band = band_from_wavelength(target_wavelength)
    print(f"Computing target PSF at λ = {target_wavelength} µm (band {target_band})")
    miri_target = stpsf.MIRI()
    miri_target.mode = 'IFU'
    miri_target.band = target_band
    miri_target.pixelscale = pixscale
    psf_target_hdu = miri_target.calc_psf(monochromatic=target_wavelength * u.micron)
    psf_target = psf_target_hdu[1].data.astype(float)
    psf_target /= psf_target.sum()      # normalise

    # Loop over slices
    for i in range(nlam):
        lam = lambdas[i]
        band = band_from_wavelength(lam)

        print(f"Processing slice {i+1}/{nlam}: λ = {lam:.3f} µm, band = {band}")

        # --- Generate native PSF for this slice ---
        miri = stpsf.MIRI()
        miri.mode = 'IFU'
        miri.band = band
        miri.pixelscale = pixscale
        psf_native_hdu = miri.calc_psf(monochromatic=lam * u.micron)
        psf_native = psf_native_hdu[1].data.astype(float)
        psf_native /= psf_native.sum()   # normalise

        # --- Compute matching kernel ---
        # Ensure both PSFs are same size; if not, pad the smaller one.
        # Here we assume they are both generated with same pixel scale and FOV,
        # but stpsf may return different array sizes for different wavelengths.
        # We'll pad to the larger size.
        h1, w1 = psf_native.shape
        h2, w2 = psf_target.shape
        h = max(h1, h2)
        w = max(w1, w2)
        psf_native_pad = np.zeros((h, w))
        psf_target_pad = np.zeros((h, w))
        # Place PSFs at centre
        y1, x1 = (h - h1)//2, (w - w1)//2
        y2, x2 = (h - h2)//2, (w - w2)//2
        psf_native_pad[y1:y1+h1, x1:x1+w1] = psf_native
        psf_target_pad[y2:y2+h2, x2:x2+w2] = psf_target

        # Compute matching kernel
        kernel = compute_matching_kernel(psf_native_pad, psf_target_pad, reg=1e-6)

        # --- Convolve flux ---
        img = np.nan_to_num(flux[i], nan=0.0)
        valid_mask = (~np.isnan(flux[i])).astype(float)

        # Use FFT convolution (kernel is same size as padded PSFs, but may be larger than image).
        # We can crop kernel to a reasonable size (e.g., to the size of the larger PSF)
        # to speed up convolution. Here we simply use fftconvolve with mode='same'.
        sm_slice = fftconvolve(img, kernel, mode='same')
        sm_slice *= valid_mask
        sm_slice[valid_mask == 0] = np.nan
        sm_flux[i] = sm_slice

        # --- Propagate errors (variance) ---
        var = np.nan_to_num(err[i]**2, nan=0.0)
        kernel_sq = kernel**2
        kernel_sq /= kernel_sq.sum()     # normalise squared kernel
        #f I dont normalise this PAHFIT breaks! for line files we wont need this

        sm_var = fftconvolve(var, kernel_sq, mode='same')
        sm_var = np.clip(sm_var, 0, None)
        sm_err[i] = np.sqrt(sm_var)
        sm_err[i][valid_mask == 0] = np.nan

    # Write output FITS file
    primary = fits.PrimaryHDU()
    sci_hdu = fits.ImageHDU(data=sm_flux.astype(np.float32), header=sci_hdr, name='SCI')
    err_hdu = fits.ImageHDU(data=sm_err.astype(np.float32), header=sci_hdr.copy(), name='ERR')
    dq_hdu  = fits.ImageHDU(data=np.zeros_like(sm_flux, dtype=np.uint32),
                            header=sci_hdr.copy(), name='DQ')
    hdul = fits.HDUList([primary, sci_hdu, err_hdu, dq_hdu])
    hdul.writeto(outfile, overwrite=True)

    print(f"Wrote {outfile}, shape={sm_flux.shape}")

# -----------------------
outdir = "./psf_smooth_cubes"
os.makedirs(outdir, exist_ok=True)

target = 17.98   # µm

for ch in range(1, 4):
    infile = f"../../github/astr796_25_V2/files/NGC253_sky_v1_17_1_ch{ch}-shortmediumlong_s3d.fits"
    #outfile = os.path.join(outdir, f"NGC253_sky_v1_17_1_ch{ch}-homogenised_{target}um.fits")
    outfile = os.path.join(outdir, f"NGC253_sky_v1_17_1_ch{ch}-shortmediumlong_s3d_smooth_3d.fits")
    smooth_cube_to_target_psf(infile, outfile, target_wavelength=target)